In [180]:
import matplotlib.pyplot as plt
import numpy as np
import os

In [225]:
root = '5_obj/'
folders = os.listdir(root)
all_alphas = {}
all_y_stars = {}
all_cm = {}
all_ass = {}
for folder in folders:
    files = os.listdir(root + folder)
    alphas = {}
    y_stars = {}
    for f in files:
        if f.startswith('correct'):
            dic = os.path.join(root,folder,f)
            cm = np.genfromtxt(dic, delimiter=',')
            cm = cm[1:,1:]
        elif f == "Assignment.csv":
            dic = os.path.join(root,folder,f)
            ass = np.genfromtxt(dic, delimiter=',')
        elif f.startswith('alpha'):
            dic = os.path.join(root,folder,f)
            t = f.split(sep='alpha')[1].split(sep='.')[0]
            alphas[t] = np.genfromtxt(dic, delimiter=',')
        elif f.startswith('y_star'):
            dic = os.path.join(root,folder,f)
            t = f.split('y_star_')[1].split('.')[0]
            y_stars[t] = np.genfromtxt(dic, delimiter=',')
#         if folder == 'csv_3_13':
            
    all_alphas[folder] = alphas
    all_y_stars[folder] = y_stars
    all_ass[folder] = ass
    all_cm[folder] = cm

In [226]:
# print(ass)
#let's get the ass format from cm
to_rem = []
for k in all_ass.keys():
    ass = all_ass[k]
    cm = all_cm[k]
#     print(k)
    if len(cm.shape) == 1:
        print(k,cm.shape)
    if len(ass.shape) == 1:
#         print(k)
        if np.nanmax(cm) +1 != len(ass):
            print(k)
            print(ass)
            print(cm)
            to_rem.append(k)
    else:
        if np.nanmax(cm) +1 != ass.shape[1]:
            print(k,' shape')
            print(ass)
            print(cm)
            to_rem.append(k)
            ass = ass[:,:np.nanmax(cm).astype('int') +1]
            all_ass[k] = ass
            
for k in to_rem:
    del all_ass[k]
    del all_cm[k]
    del all_y_stars[k]
    del all_alphas[k]


In [227]:
k = 'csv_5_13'
dists = 0
nums = 0
for k in all_cm:
    num_alphs = np.nanmax(all_cm[k])+1
    num_alphs = num_alphs.astype('int')
    new_cm = np.zeros((num_alphs,num_alphs))
    all_cm[k]
    for i in range(all_cm[k].shape[0]):
        for j in range(all_cm[k].shape[1]-1):
            if not np.isnan(all_cm[k][i,j+1]):
                nx = all_cm[k][i,j+1].astype('int')
                bf = all_cm[k][i,j].astype('int')
                new_cm[bf,nx] = 1
    #sometimes the shape is 1, othertimes is two
    if len(all_ass[k].shape) == 1:
        all_ass[k] = all_ass[k][np.newaxis,:]
    new_ass = all_ass[k]
    new_ass = np.pad(new_ass,pad_width=((0,num_alphs-new_ass.shape[0]),(0,0)),mode='constant')
    temp = new_ass-new_cm
    false_negatives = np.sum(temp[temp<0]**2)
    false_positives = np.sum(temp[temp>0]**2)
    true_positives = np.sum(new_ass)-false_positives
#     print(new_ass)
#     print(new_cm)
#     print(temp)
    temp = np.sum(temp**2,axis=1)
#     print(temp)
    temp = len(np.where(temp==0)[0])
#     print(temp)
#     print(false_negatives,false_positives,true_positives)
    dists+=temp
    nums+= num_alphs
dists/nums


0.8477653631284916

In [81]:
t = np.arange(10).reshape((5,2))
x = np.arange(10).reshape((5,2))+5
t = np.array([0,0,1,-2,0])
np.where(t!=0)

(array([2, 3]),)